In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

#### Import model, training function 

In [2]:
from annsa.load_dataset import dataset_to_spectrakeys
from annsa.model_classes import build_dnn_model, compile_model
from annsa.load_pretrained_network import load_features

Using TensorFlow backend.


## Dataset Construction

#### Load dataset

In [29]:
dataset = np.load(os.path.join('dataset_generation',
                               'uenrichment_dataset_100000.npy'))
training_spectra = np.array(dataset.item()['spectra'], dtype='float64')
all_keys = np.array(dataset.item()['keys'])

## Model Construction

In [30]:
architecture_id = os.path.join('..',
                               'source-interdiction',
                               'final_training_notebooks',
                               'final-models',
                               'learningcurve-dnn-full-final-features',
                              )

In [31]:
model_features = load_features(architecture_id)
model_features.input_dim = 1024
model_features.dropout_rate = model_features.dropout_probability
model_features.loss = tf.keras.losses.mean_squared_error
model_features.optimizer = tf.keras.optimizers.Adam
model_features.output_function = tf.nn.sigmoid
model_features.output_size = 1
model_features.metrics = ['mse']
model_features.learning_rate = model_features.learining_rate

In [32]:
dae_model = load_model('../source-interdiction/final_training_notebooks/final-models-keras/daepretrain-full-final.hdf5')

## Train network

In [33]:
num_epochs = 200
patience = 10

model_id_save_as = 'daednn-full'

earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_mean_squared_error',
    patience=patience,
    mode='min',
    min_delta=0.01,
    restore_best_weights=True)

## Scale input data

In [34]:
training_spectra_scaled = model_features.scaler.transform(training_spectra)

# Train

In [35]:
kf = KFold(n_splits=5)

k = 0
for train_index, validation_index in kf.split(training_spectra_scaled):
    model = compile_model(
        build_dnn_model,
        model_features)
    model_weights = model.get_weights()
    model_weights_updated = model_weights[:]
    model_weights_updated[0:2] = dae_model.get_weights()[0:2]
    model.set_weights(model_weights_updated)

    csv_logger = tf.keras.callbacks.CSVLogger(
        os.path.join('final-models-keras/',
        model_id_save_as+'_fold' + str(k) + '.log')

    output = model.fit(
        x=training_spectra_scaled[train_index],
        y=all_keys[train_index],
        epochs=num_epochs,
        verbose=1,
        validation_data=(training_spectra_scaled[validation_index],
                         all_keys[validation_index]),
        shuffle=True,
        callbacks=[earlystop_callback, csv_logger],
    )
    model.save(
        os.path.join('final-models-keras',
                     model_id_save_as + '_fold' + str(k) + '.hdf5')
    k += 1

Train on 80000 samples, validate on 20000 samples
Epoch 1/200
80000/80000 [==============================] - 11s 141us/sample - loss: 4.3937 - mean_squared_error: 0.2641 - val_loss: 0.4282 - val_mean_squared_error: 0.3325
Epoch 2/200
80000/80000 [==============================] - 10s 131us/sample - loss: 0.3121 - mean_squared_error: 0.2817 - val_loss: 0.2099 - val_mean_squared_error: 0.1851
Epoch 3/200
80000/80000 [==============================] - 11s 132us/sample - loss: 0.1604 - mean_squared_error: 0.1434 - val_loss: 0.0553 - val_mean_squared_error: 0.0439
Epoch 4/200
80000/80000 [==============================] - 10s 131us/sample - loss: 0.0476 - mean_squared_error: 0.0407 - val_loss: 0.0436 - val_mean_squared_error: 0.0384
Epoch 5/200
80000/80000 [==============================] - 10s 130us/sample - loss: 0.0415 - mean_squared_error: 0.0358 - val_loss: 0.0341 - val_mean_squared_error: 0.0287
Epoch 6/200
80000/80000 [==============================] - 10s 130us/sample - loss: 0.0393